# Loan Amount Prediction - Utkarsh Gaikwad

### Importing Dataset 

In [105]:
from warnings import filterwarnings
filterwarnings('ignore')
import pandas as pd
training = pd.read_csv("D:/Etlhive class data/Project Loan 280822 Classification and Regression/Input/training_set.csv")
testing = pd.read_csv("D:/Etlhive class data/Project Loan 280822 Classification and Regression/Input/testing_set.csv")
training.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


In [106]:
training.nunique()

Loan_ID              614
Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      504
CoapplicantIncome    286
LoanAmount           203
Loan_Amount_Term      10
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

In [107]:
training.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome      float64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

# Converting Credit History to Object datatype 

In [108]:
crd_hst = []
for i in range(len(training)):
    if training['Credit_History'][i]==1:
        crd_hst.append('Yes')
    elif training['Credit_History'][i]==0:
        crd_hst.append('No')
    else:
        crd_hst.append(None)
training['Credit_History']=crd_hst

In [109]:
crd_hst1 = []
for i in range(len(testing)):
    if testing['Credit_History'][i]==1:
        crd_hst1.append('Yes')
    elif testing['Credit_History'][i]==0:
        crd_hst1.append('No')
    else:
        crd_hst1.append(None)
testing['Credit_History']=crd_hst1

In [110]:
import import_ipynb
from PM3 import replacer
replacer(training)
replacer(testing)

Missing Values replaced in DataFrame
Missing Values replaced in DataFrame


# Defining X, Y

In [111]:
X = training.drop(labels=['Loan_ID','LoanAmount'],axis=1)
Y = training[['LoanAmount']]

In [112]:
from PM3 import catcon
cat, con = catcon(X)

Categorical and Continuous variables appended


# Preprocessing

In [113]:
X1 = X[con]
X2 = pd.get_dummies(X[cat])
Xnew = X1.join(X2)

# Random Forest checking

In [114]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=21)

tg = {"n_estimators":[24,25,26],
      "max_depth":[9,10,11,12],
      "min_samples_split":[2,5,10],
      "min_samples_leaf":[2,5,10],
      "criterion":['mae','mse']}
cv = GridSearchCV(rfr,tg,scoring='neg_mean_absolute_error',cv=4,verbose=1)
cvmodel = cv.fit(Xnew,Y)
cvmodel.best_params_

Fitting 4 folds for each of 216 candidates, totalling 864 fits


{'criterion': 'mae',
 'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 25}

In [115]:
model1 = cvmodel.best_estimator_
pred = model1.predict(Xnew)
from sklearn.metrics import mean_squared_error, r2_score
err = mean_squared_error(pred,Y)
r2 = r2_score(pred,Y)
print('Mean Squared Error is      : ',round(err,2))
print('Root Mean Squared Error is : ',round(err**0.5,2))
print('R2 Score is                : ',round(r2,4))

Mean Squared Error is      :  1358.75
Root Mean Squared Error is :  36.86
R2 Score is                :  0.6711


In [116]:
Q = pd.DataFrame(Xnew.columns,columns=["col"])
Q['imp']=model1.feature_importances_

In [117]:
Q.sort_values(by=["imp"],ascending=False)

,col,imp
0,ApplicantIncome,0.516764
1,CoapplicantIncome,0.239984
2,Loan_Amount_Term,0.026147
7,Dependents_0,0.025285
18,Property_Area_Semiurban,0.021315
19,Property_Area_Urban,0.020144
10,Dependents_3+,0.019131
9,Dependents_2,0.015252
17,Property_Area_Rural,0.013290
20,Loan_Status_N,0.012914


# considering top 3 predictors

In [118]:
Xnew2 = training[['ApplicantIncome','CoapplicantIncome','Loan_Amount_Term']]
Y = training[['LoanAmount']]

In [119]:
rfr = RandomForestRegressor(random_state=21)

tg = {"n_estimators":[2,5,10,20],
      "max_depth":[2,5,10,20],
      "min_samples_split":[2,5,10,20],
      "min_samples_leaf":[2,5,10,20],
      "criterion":['mae','mse']}
cv = GridSearchCV(rfr,tg,scoring='neg_mean_absolute_error',cv=4,verbose=1)
cvmodel = cv.fit(Xnew2,Y)
cvmodel.best_params_

Fitting 4 folds for each of 512 candidates, totalling 2048 fits


{'criterion': 'mse',
 'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 20,
 'n_estimators': 20}

In [120]:
model2 = cvmodel.best_estimator_
pred = model2.predict(Xnew2)
from sklearn.metrics import mean_squared_error, r2_score
err = mean_squared_error(pred,Y)
r2 = r2_score(pred,Y)
print('Mean Squared Error is      : ',round(err,2))
print('Root Mean Squared Error is : ',round(err**0.5,2))
print('R2 Score is                : ',round(r2,4))

Mean Squared Error is      :  2469.91
Root Mean Squared Error is :  49.7
R2 Score is                :  0.3647


# Removing Predictors reduced R2 Score

# Checking for Adaboost

In [121]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=2)
from sklearn.model_selection import GridSearchCV
abr = AdaBoostRegressor(dtr)

tg = {'n_estimators':range(1,41)}
cv = GridSearchCV(abr,tg,scoring='neg_mean_absolute_error',cv=20)
cvmodel = cv.fit(Xnew,Y)
cvmodel.best_params_

{'n_estimators': 4}

In [122]:
model3 = cvmodel.best_estimator_
pred = model3.predict(Xnew)
from sklearn.metrics import mean_squared_error, r2_score
err = mean_squared_error(pred,Y)
r2 = r2_score(pred,Y)
print('Mean Squared Error is      : ',round(err,2))
print('Root Mean Squared Error is : ',round(err**0.5,2))
print('R2 Score is                : ',round(r2,4))

Mean Squared Error is      :  3742.89
Root Mean Squared Error is :  61.18
R2 Score is                :  -0.4415


# Adaboost has bad R2 Score

# Checking with Gaussian Process regression

### test_train split

In [123]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

### Model Creation

In [124]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,ExpSineSquared, DotProduct,ConstantKernel)
                                              
# Determination of the kernel used and the value lenght_scale at which the optimization process starts
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3))

# Application of the regressor (alpha makes a statement about a possible noise of the training data)
gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1)
gp.fit(xtrain, ytrain)

# Prediction of the dependent variable for the dataset
tr_pred, tr_sigma = gp.predict(xtrain, return_std=True)
ts_pred, ts_sigma = gp.predict(xtest, return_std=True)

In [125]:
from sklearn.metrics import mean_squared_error, r2_score
tr_err = mean_squared_error(tr_pred,ytrain)
ts_err = mean_squared_error(ts_pred,ytest)
tr_r2 = r2_score(tr_pred,ytrain)
ts_r2 = r2_score(ts_pred,ytest)
print('Training Mean Squared Error is : ',round(tr_err,2))
print('Testing Mean Squared Error is  : ',round(ts_err,2))
print('Training R2 Score is           : ',round(tr_r2,4))
print('Testing R2 Score is            : ',round(ts_r2,4))

Training Mean Squared Error is :  0.8
Testing Mean Squared Error is  :  31852.8
Training R2 Score is           :  0.9999
Testing R2 Score is            :  -18.9931


# Hence Gaussian Process regressor fails in testing 

# Checking KNN with weights

In [126]:
X = training.drop(labels=['LoanAmount','Loan_ID'],axis=1)
Y = training[['LoanAmount']]

In [127]:
cat,con = catcon(X)

Categorical and Continuous variables appended


In [128]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X11 = pd.DataFrame(ss.fit_transform(X[con]),columns=con)

In [129]:
X21 = pd.get_dummies(X[cat])
Xnew3 = X11.join(X21)

In [130]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

tg = {"n_neighbors":range(2,51,1),"weights":['uniform','distance']}
cv = GridSearchCV(knn,tg,scoring='neg_mean_absolute_error',cv=20)
cvmodel = cv.fit(Xnew3,Y)
cvmodel.best_params_

{'n_neighbors': 36, 'weights': 'distance'}

In [131]:
model4 = cvmodel.best_estimator_
pred = model4.predict(Xnew3)
from sklearn.metrics import mean_squared_error, r2_score
err = mean_squared_error(pred,Y)
r2 = r2_score(pred,Y)
print('Mean Squared Error is      : ',round(err,4))
print('Root Mean Squared Error is : ',round(err**0.5,4))
print('R2 Score is                : ',round(r2,6))

Mean Squared Error is      :  0.6384
Root Mean Squared Error is :  0.799
R2 Score is                :  0.999909


# Weighted KNN has highest score

### Filtering Out Rejected Loans

In [132]:
Xnew3.head()

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,...,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Credit_History_No,Credit_History_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
0,0.072651,-0.554368,0.279851,0,1,1,0,1,0,0,...,0,1,0,0,1,0,0,1,0,1
1,0.000000,-0.038610,0.279851,0,1,0,1,0,1,0,...,0,1,0,0,1,1,0,0,1,0
2,-0.394096,-0.554368,0.279851,0,1,0,1,1,0,0,...,0,0,1,0,1,0,0,1,0,1
3,-0.462412,0.252103,0.279851,0,1,0,1,1,0,0,...,1,1,0,0,1,0,0,1,0,1
4,0.097389,-0.554368,0.279851,0,1,1,0,1,0,0,...,0,1,0,0,1,0,0,1,0,1


In [133]:
X_Loan_status_no = Xnew3[Xnew3['Loan_Status_N']==1]
X_Loan_status_no.head()

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,...,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Credit_History_No,Credit_History_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
1,0.000000,-0.038610,0.279851,0,1,0,1,0,1,0,...,0,1,0,0,1,1,0,0,1,0
7,-0.388198,0.302037,0.279851,0,1,0,1,0,0,0,...,0,1,0,1,0,0,1,0,1,0
9,1.218139,3.196849,0.279851,0,1,0,1,0,1,0,...,0,1,0,0,1,0,1,0,1,0
13,-0.582007,0.416954,0.279851,0,1,1,0,1,0,0,...,0,1,0,0,1,1,0,0,1,0
17,-0.310543,-0.554368,0.279851,1,0,1,0,1,0,0,...,0,1,0,1,0,0,0,1,1,0


# Converting No Loan Status to Yes and predicting LoanAmount

In [134]:
X_Loan_status_no['Loan_Status_N']=0
X_Loan_status_no['Loan_Status_Y']=1

In [135]:
pred_no = model4.predict(X_Loan_status_no)

In [136]:
X_Loan_status_no['Predicted_LoanAmount']=pred_no

In [137]:
X_Loan_status_no

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,...,Self_Employed_No,Self_Employed_Yes,Credit_History_No,Credit_History_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y,Predicted_LoanAmount
1,0.000000,-0.038610,0.279851,0,1,0,1,0,1,0,...,1,0,0,1,1,0,0,0,1,146.418919
7,-0.388198,0.302037,0.279851,0,1,0,1,0,0,0,...,1,0,1,0,0,1,0,0,1,137.601130
9,1.218139,3.196849,0.279851,0,1,0,1,0,1,0,...,1,0,0,1,0,1,0,0,1,254.876863
13,-0.582007,0.416954,0.279851,0,1,1,0,1,0,0,...,1,0,0,1,1,0,0,0,1,126.865639
17,-0.310543,-0.554368,0.279851,1,0,1,0,1,0,0,...,1,0,1,0,0,0,1,0,1,90.214295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.160136,-0.212353,0.279851,0,1,0,1,0,0,1,...,0,1,0,1,1,0,0,0,1,127.587417
597,-0.396226,-0.554368,0.279851,0,1,1,0,1,0,0,...,1,0,1,0,0,1,0,0,1,99.937257
600,-0.817429,13.696358,-2.518655,1,0,1,0,0,0,0,...,1,0,0,1,0,0,1,0,1,232.797686
605,-0.492393,0.745288,-2.518655,0,1,0,1,1,0,0,...,1,0,0,1,0,0,1,0,1,106.237355


In [138]:
X_no = training[training['Loan_Status']=='N']
X_no.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.0,360.0,Yes,Rural,N
7,LP001014,Male,Yes,3+,Graduate,No,3036.00000,2504.0,158.0,360.0,No,Semiurban,N
9,LP001020,Male,Yes,1,Graduate,No,12841.00000,10968.0,349.0,360.0,Yes,Semiurban,N
13,LP001029,Male,No,0,Graduate,No,1853.00000,2840.0,114.0,360.0,Yes,Rural,N
17,LP001036,Female,No,0,Graduate,No,3510.00000,0.0,76.0,360.0,No,Urban,N


In [139]:
X_no['Predicted_LoanAmount']=pred_no

In [142]:
X_no

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Predicted_LoanAmount
1,LP001003,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.000000,360.0,Yes,Rural,N,146.418919
7,LP001014,Male,Yes,3+,Graduate,No,3036.00000,2504.0,158.000000,360.0,No,Semiurban,N,137.601130
9,LP001020,Male,Yes,1,Graduate,No,12841.00000,10968.0,349.000000,360.0,Yes,Semiurban,N,254.876863
13,LP001029,Male,No,0,Graduate,No,1853.00000,2840.0,114.000000,360.0,Yes,Rural,N,126.865639
17,LP001036,Female,No,0,Graduate,No,3510.00000,0.0,76.000000,360.0,No,Urban,N,90.214295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,LP002941,Male,Yes,2,Not Graduate,Yes,6383.00000,1000.0,187.000000,360.0,Yes,Rural,N,127.587417
597,LP002943,Male,No,0,Graduate,No,2987.00000,0.0,88.000000,360.0,No,Semiurban,N,99.937257
600,LP002949,Female,No,3+,Graduate,No,416.00000,41667.0,350.000000,180.0,Yes,Urban,N,232.797686
605,LP002960,Male,Yes,0,Not Graduate,No,2400.00000,3800.0,146.412162,180.0,Yes,Urban,N,106.237355


In [141]:
X_no.to_csv("D:/Etlhive class data/Project Loan 280822 Classification and Regression/Output/PredictedLoanAmount.csv")